In [1]:
!pip install transformers -U

You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [2]:
pip install spacy

You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
!python -m spacy download ru_core_news_sm

     |████████████████████████████████| 16.4 MB 1.7 MB/s eta 0:00:01
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [4]:
!pip install pymorphy2[fast]

You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [5]:
!git clone  https://github.com/sberbank-ai/ru-gpts

fatal: destination path 'ru-gpts' already exists and is not an empty directory.


In [6]:
!git clone https://github.com/huggingface/transformers.git

fatal: destination path 'transformers' already exists and is not an empty directory.


In [7]:
!pip install natasha

You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [8]:
from natasha import (   
    MorphVocab,
    NewsNERTagger,
    NamesExtractor,
    MorphVocab,
    Segmenter, 
    NewsEmbedding,
    PER,
    Doc,
    ORG
)

In [9]:
from transformers import (GPT2LMHeadModel, GPT2Tokenizer,
                          TextDataset, DataCollatorForLanguageModeling,
                          Trainer, TrainingArguments, AutoModelWithLMHead, pipeline)

In [10]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import re

In [11]:
import torch
import pandas as pd

In [12]:
import numpy as np

In [13]:
import pymorphy2

In [14]:
import spacy

**Открываем файл с текстами, которые скачал скрапер. Размер файла не большой, что позволяет считать все в одну переменную.**

In [15]:
with open('curr_congrats.txt') as handler:
    full_text = handler.read()
full_text[:1000]

'\nTITLE\nЭдварду Радзинскому, писателю, драматургу, сценаристу\nCONTENT\n Уважаемый Эдвард Станиславович!Примите поздравления с\xa085-летием.Человек многогранного дарования и\xa0большой созидательной энергии, Вы по\xa0праву входите в\xa0число признанных\nдраматургов, мастеров современного театрального искусства. И\xa0конечно, Вас\nхорошо знают и\xa0любят как автора целого ряда просветительских проектов, чьё\nтворчество завоевало сердца самой широкой аудитории, в\xa0том числе молодёжи.Желаю доброго здоровья, благополучия и\xa0бодрости\nдуха. Владимир Путин     Статус материала Опубликован в\xa0разделе: Телеграммы  Ссылка на материал: kremlin.ru/d/66751   Текстовая версия      \nEND\n\nTITLE\nКоллективу радиостанции «Орфей»\nCONTENT\n Дорогие друзья!Поздравляю вас со\xa0знаменательным юбилеем\xa0– 30-летием радиостанции «Орфей».Все эти годы ваш коллектив объединяет настоящих единомышленников, профессионалов, преданных высокому искусству, стремящихся внести свой вклад в\xa0сбережение луч

**Поздравления разделены разметкой на заголовки и сами поздравления. Заголовок содержит адресат (кого поздравляют) и выделен разметкой "TITLE". Разметка "CONTENT" отделяет содержание поздравления.**
*(Это не использовалось при обучении, но теоретически можно было, поэтому разделение осталось.)*

**Аннотация "END" обозначает конец целого поздравления ("TITLE" + "CONTENT").**

In [16]:
titles_list = []
congrats_list = []

for letter in full_text.split('END'):
    letter = letter.replace('\n', '').replace('TITLE', '')

    if len(letter.split('CONTENT')) > 1:
        titles_list.append(letter.split('CONTENT')[0])
        congrats_list.append(letter.split('CONTENT')[1])

print("Сопоставим адресат и поздравление:\n")
print(titles_list[:1])
print(congrats_list[:1])
print("\nПроверка количества адресатов", len(titles_list))
print("Проверка количества поздравлений", len(congrats_list))

Сопоставим адресат и поздравление:

['Эдварду Радзинскому, писателю, драматургу, сценаристу']
[' Уважаемый Эдвард Станиславович!Примите поздравления с\xa085-летием.Человек многогранного дарования и\xa0большой созидательной энергии, Вы по\xa0праву входите в\xa0число признанныхдраматургов, мастеров современного театрального искусства. И\xa0конечно, Васхорошо знают и\xa0любят как автора целого ряда просветительских проектов, чьётворчество завоевало сердца самой широкой аудитории, в\xa0том числе молодёжи.Желаю доброго здоровья, благополучия и\xa0бодростидуха. Владимир Путин     Статус материала Опубликован в\xa0разделе: Телеграммы  Ссылка на материал: kremlin.ru/d/66751   Текстовая версия      ']

Проверка количества адресатов 888
Проверка количества поздравлений 888


**Сделаем датасет и добавим в него колонку с суммарными данными:**

In [17]:
data = {'Title': titles_list,
        'Content': congrats_list}

my_df = pd.DataFrame(data)
my_df['Sum'] = [my_df['Title'].tolist()[i] + my_df['Content'].tolist()[i]
                for i in range(len(my_df))]
my_df

,Title,Content,Sum
0,"Эдварду Радзинскому, писателю, драматургу, сце...",Уважаемый Эдвард Станиславович!Примите поздра...,"Эдварду Радзинскому, писателю, драматургу, сце..."
1,Коллективу радиостанции «Орфей»,Дорогие друзья!Поздравляю вас со знаменательн...,Коллективу радиостанции «Орфей» Дорогие друзья...
2,"Михаилу Ковальчуку, президенту НИЦ «Курчатовск...",Уважаемый Михаил Валентинович!Примите мои поз...,"Михаилу Ковальчуку, президенту НИЦ «Курчатовск..."
3,"Ирине Андреевой и Ивану Штылю, победителям чем...",Уважаемые Ирина Александровна и Иван Александ...,"Ирине Андреевой и Ивану Штылю, победителям чем..."
4,Работникам и ветеранам оборонно-промышленного ...,Уважаемые друзья!Поздравляю вас с Днём оружей...,Работникам и ветеранам оборонно-промышленного ...
...,...,...,...
883,Жителям Республики Мордовия,Дорогие друзья!Поздравляю вас с юбилеем – 75-...,Жителям Республики Мордовия Дорогие друзья!Поз...
884,Коллективу Института физических проблем им.П.Л...,Поздравляю коллектив института с 70-летием со...,Коллективу Института физических проблем им.П.Л...
885,Жителям Красноярского края,Уважаемые друзья!Поздравляю вас с 70-летием с...,Жителям Красноярского края Уважаемые друзья!По...
886,В.А.ЛОГИНОВУ,Уважаемый Владимир Александрович!Поздравляю В...,В.А.ЛОГИНОВУ Уважаемый Владимир Александрович!...


**Очистим суммарную колонку от лишних символов**

In [18]:
new_sum = []

for summary in my_df["Sum"]:
    # print(summary)
    summary = str(summary).strip()
    summary = re.sub(r"\s", " ", summary)
    summary = re.sub(r"\n", "", summary)
    summary = re.sub(r"Статус материала Опубликован в разделе: Телеграммы", "", summary)
    summary = re.sub(r"Ссылка на материал:", "", summary)
    summary = re.sub(r"Текстовая версия:", "", summary)
    summary = re.sub(r"Ссылка на материал:", "", summary)
    summary = re.sub(r"([a-zA-Z]+)", "", summary)
    summary = re.sub(r"([/*#@+]+)", "", summary)
    summary = re.sub(r"Текстовая версия", "", summary)

    new_sum.append(summary)

my_df["Sum"] = new_sum

**Маскируем полное поздравление вместе с адресатом, возьмем имена и организации. Сначала у меня была идея вместе с символом MASK вставлять падеж, чтобы модель обучилась на конкретных падежах, и потом можно было просто заменить разметку MASK с соответствующим падежом на именованную сущность в этом падеже (типа MASKnomn - значит падеж будет именительный).**

**Из этой идеи ничего не вышло, я подозреваю, что это из-за посимвольной токенизации внутри токенизатора библиотеки Transformers. Однако, реализацию я решил оставить.**

In [19]:
morph = pymorphy2.MorphAnalyzer()
nname = morph.parse('Владимира')[0]
print(nname)
str(nname[1]).split(',')[-1]

Parse(word='владимира', tag=OpencorporaTag('NOUN,anim,masc,Name sing,gent'), normal_form='владимир', score=0.682926, methods_stack=((DictionaryAnalyzer(), 'владимира', 27, 1),))


'gent'

In [20]:
emb = NewsEmbedding()
morph_vocab = MorphVocab()
names_extractor = NamesExtractor(morph_vocab)
ner_tagger = NewsNERTagger(emb)
segmenter = Segmenter()

In [21]:
masked_content = []
morph = pymorphy2.MorphAnalyzer()

for sent in my_df['Sum']:
    doc = Doc(sent)
    doc.segment(segmenter)
    doc.tag_ner(ner_tagger)

    for span in doc.spans:
        if span.type == PER or span.type == ORG:
            morph_name = morph.parse(span.text)[0]
            morph_case = str(morph_name[1]).split(',')[-1]
            sent = sent.replace(span.text, f'MASK{morph_case}')

    masked_content.append(sent)

my_df['Masked_sum'] = masked_content
my_df

,Title,Content,Sum,Masked_sum
0,"Эдварду Радзинскому, писателю, драматургу, сце...",Уважаемый Эдвард Станиславович!Примите поздра...,"Эдварду Радзинскому, писателю, драматургу, сце...","MASKdatv, писателю, драматургу, сценаристу MAS..."
1,Коллективу радиостанции «Орфей»,Дорогие друзья!Поздравляю вас со знаменательн...,Коллективу радиостанции «Орфей» Дорогие друзья...,Коллективу радиостанции «MASKnomn» Дорогие дру...
2,"Михаилу Ковальчуку, президенту НИЦ «Курчатовск...",Уважаемый Михаил Валентинович!Примите мои поз...,"Михаилу Ковальчуку, президенту НИЦ «Курчатовск...","MASKdatv, президенту MASKUNKN MASKnomn!Примите..."
3,"Ирине Андреевой и Ивану Штылю, победителям чем...",Уважаемые Ирина Александровна и Иван Александ...,"Ирине Андреевой и Ивану Штылю, победителям чем...","MASKgent и MASKdatv, победителям чемпионата ми..."
4,Работникам и ветеранам оборонно-промышленного ...,Уважаемые друзья!Поздравляю вас с Днём оружей...,Работникам и ветеранам оборонно-промышленного ...,Работникам и ветеранам оборонно-промышленного ...
...,...,...,...,...
883,Жителям Республики Мордовия,Дорогие друзья!Поздравляю вас с юбилеем – 75-...,Жителям Республики Мордовия Дорогие друзья!Поз...,Жителям Республики Мордовия Дорогие друзья!Поз...
884,Коллективу Института физических проблем им.П.Л...,Поздравляю коллектив института с 70-летием со...,Коллективу Института физических проблем им.П.Л...,Коллективу MASKindc коллектив института с 70-л...
885,Жителям Красноярского края,Уважаемые друзья!Поздравляю вас с 70-летием с...,Жителям Красноярского края Уважаемые друзья!По...,Жителям Красноярского края Уважаемые друзья!По...
886,В.А.ЛОГИНОВУ,Уважаемый Владимир Александрович!Поздравляю В...,В.А.ЛОГИНОВУ Уважаемый Владимир Александрович!...,MASKnomn!Поздравляю Вас с официальным вступлен...


**Разобьем датасет на наборы**

In [22]:
train_df, validate_df = np.split(
    my_df.sample(frac=1, random_state=42),
    [int(.8*len(my_df))])

In [23]:
len(train_df), len(validate_df)

(710, 178)

**Сделаем текстовые файлы для обучения модели**

In [24]:
!rm train.txt valid.txt

In [25]:
def make_file_from_series(filename, series):
    with open(filename, 'w') as f:
        for entry in series:
            f.write(entry)

In [26]:
make_file_from_series('train.txt', train_df['Masked_sum'])
make_file_from_series('valid.txt', validate_df['Masked_sum'])

In [27]:
!ls 

Kutsev.ipynb				    curr_congrats.txt
Untitled.ipynb				    gpt-letters
cached_lm_GPT2Tokenizer_128_test.txt	    ru-gpts
cached_lm_GPT2Tokenizer_128_test.txt.lock   train.txt
cached_lm_GPT2Tokenizer_128_train.txt	    transformers
cached_lm_GPT2Tokenizer_128_train.txt.lock  valid.txt
cached_lm_GPT2Tokenizer_128_valid.txt	    work
cached_lm_GPT2Tokenizer_128_valid.txt.lock


In [28]:
with open('train.txt') as handler:
    full_text = handler.read()
full_text[:3000]

'MASKloct, победительнице чемпионата мира по лёгкой атлетике 2019 года в Дохе (Катар) в прыжках в высоту Уважаемая MASKnomn!Поздравляю Вас с триумфальным выступлением в Дохе.Вы вновь ярко продемонстрировали впечатляющий спортивный, победный потенциал, мастерство и сильный характер. Покорив заданную высоту, по праву стали обладательницей золотой медали, реализовали заветную мечту – заслужили звание трёхкратной чемпионки мира. И, конечно, Ваш успех – это отличный подарок для нашей сборной, тренеров и наставников, многочисленных болельщиков.От души желаю Вам новых свершений и всего самого доброго. MASKnomn        .61696   Коллективу «MASKgent» Дорогие друзья!Поздравляю вас со знаменательной датой – 25-летиемсоздания «MASKgent».За прошедшие годы газета стала одним из ведущихпериодических изданий страны, завоевала уважение большой читательской аудиториии авторитет в отечественном медиасообществе. Её ценят за информационнуюнаполненность, глубокий подход к освещению важнейших тем и событий, з

**Приведем данные в подходящий для обучения модели вид**

In [29]:
tokenizer = GPT2Tokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")
train_path = 'train.txt'
valid_path = 'valid.txt'

In [32]:
def load_dataset(train_path, valid_path, tokenizer):

    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
    
    valid_dataset = TextDataset(
      tokenizer=tokenizer,
      file_path=valid_path,
      block_size=128)
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset, valid_dataset, data_collator

train_dataset, valid_dataset, data_collator = load_dataset(train_path, valid_path, tokenizer)

**Параметры обучения модели**

In [69]:
model = GPT2LMHeadModel.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")


training_args = TrainingArguments(
    output_dir="./gpt-letters",
    overwrite_output_dir=True,
    num_train_epochs=650,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    eval_steps=400,
    save_steps=150,
    warmup_steps=500,
    prediction_loss_only=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
)

loading configuration file https://huggingface.co/sberbank-ai/rugpt3small_based_on_gpt2/resolve/main/config.json from cache at /home/mdurinov/.cache/huggingface/transformers/06f48b6b3173390d047e15d691fda67ae4ea7733a5eea4b6e0115f5099c4e700.b5cdfa39c63384f94159c36bc9042660c747cea5cf520b43d543bd2c68b3164d
Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 2048,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 2048,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "sum

На всякий случай удалю папку с предыдущей моделью.

In [70]:
!rm -rf gpt-letters

In [71]:
trainer.train()

***** Running training *****
  Num examples = 1048
  Num Epochs = 650
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 21450


Step,Training Loss
500,1.898700
1000,0.630300
1500,0.201100
2000,0.079100
2500,0.045600
3000,0.033400
3500,0.027600
4000,0.023900
4500,0.021000
5000,0.019600


Saving model checkpoint to ./gpt-letters/checkpoint-150
Configuration saved in ./gpt-letters/checkpoint-150/config.json
Model weights saved in ./gpt-letters/checkpoint-150/pytorch_model.bin
Saving model checkpoint to ./gpt-letters/checkpoint-300
Configuration saved in ./gpt-letters/checkpoint-300/config.json
Model weights saved in ./gpt-letters/checkpoint-300/pytorch_model.bin
Saving model checkpoint to ./gpt-letters/checkpoint-450
Configuration saved in ./gpt-letters/checkpoint-450/config.json
Model weights saved in ./gpt-letters/checkpoint-450/pytorch_model.bin
Saving model checkpoint to ./gpt-letters/checkpoint-600
Configuration saved in ./gpt-letters/checkpoint-600/config.json
Model weights saved in ./gpt-letters/checkpoint-600/pytorch_model.bin
Saving model checkpoint to ./gpt-letters/checkpoint-750
Configuration saved in ./gpt-letters/checkpoint-750/config.json
Model weights saved in ./gpt-letters/checkpoint-750/pytorch_model.bin
Saving model checkpoint to ./gpt-letters/checkpoin

TrainOutput(global_step=21450, training_loss=0.07872538582150475, metrics={'train_runtime': 2702.4303, 'train_samples_per_second': 252.069, 'train_steps_per_second': 7.937, 'total_flos': 4.44980330496e+16, 'train_loss': 0.07872538582150475, 'epoch': 650.0})

In [72]:
trainer.save_model()

Saving model checkpoint to ./gpt-letters
Configuration saved in ./gpt-letters/config.json
Model weights saved in ./gpt-letters/pytorch_model.bin


**Генерация текстов. Выберем пайплайн с моделью для генерации. Для удобства я ограничиваю максимальную длину, но на первый вариантах генерации она была конечна, по крайней мере на нескольких генерациях (я сверял длины как строк, так и количества слов, они различались).**

In [73]:
local_generator = pipeline('text-generation', model='./gpt-letters',
                           tokenizer=tokenizer, max_length=500)

loading configuration file ./gpt-letters/config.json
Model config GPT2Config {
  "_name_or_path": "./gpt-letters",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 2048,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 2048,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "torch_dtype": "float32",
  "transformers_version": "4.15.0",
  "use_cache": true,
  "vocab_size": 50264
}

loading configuration file ./gpt-letters/config.json
Model config GPT2Config {
  "_

**Для замены разметки MASK на соответствующий падеж я использовал сочетание библиотек Spacy и Pymorphy (можно бы, наверное, было все сделать на spacy, но я не нашел, как там преобразовать сущность к требуемому падежу).**

**Однако, с помощью синтаксического парсера Spacy можно сравнительно удобно извлекать главное слово/зависимые слова, и ориентироваться на их падеж, чтобы проставить паджеж при замене токена MASK.**
**Вот пример такого извлечения:**

In [74]:
nlp = spacy.load("ru_core_news_sm")

text = """MASK, учёному в области молекулярной генетики, академику MASK MASK!
          Примите мои поздравления с -летием и наилучшие пожелания.
          MASK посвятили себя науке и посвятили себя воспитанию молодёжи.
          Ваши исследования и разработки в области молекулярной генетики стали
          настоящим прорывом в науке и медицине.
          Здоровья Вам, благополучия и всего самого доброго
          Президенту Республики Беларусь Уважаемый MASK"""

text = re.sub("(MASK ?)+", 'MASK', text)
doc = nlp(text)

print(doc.text)
for token in doc:
    if 'MASK' in token.text:
        print(token.text,
              [child.morph.to_dict().get('Case') for child in token.children
               if child.morph.to_dict().get('Case')],
              token.head.morph.to_dict().get('Case'),
              [i for i in token.ancestors])

MASK, учёному в области молекулярной генетики, академику MASK!
          Примите мои поздравления с -летием и наилучшие пожелания.
          MASKпосвятили себя науке и посвятили себя воспитанию молодёжи.
          Ваши исследования и разработки в области молекулярной генетики стали
          настоящим прорывом в науке и медицине.
          Здоровья Вам, благополучия и всего самого доброго
          Президенту Республики Беларусь Уважаемый MASK
MASK ['Dat', 'Dat'] None []
MASK [] Dat [академику, MASK]
MASKпосвятили ['Acc', 'Dat'] None []
MASK ['Nom'] None []


**Функция для преобразования падежа с помощью Pymorphy. Принимает на вход полное имя и наименование падежа в аннотации для Pymorphy. Возвращает имя в требуемом падеже.**

In [75]:
def case_name(full_name, case):
    morph = pymorphy2.MorphAnalyzer()
    new_name = ''
    for part in full_name.split():
        morph_part = morph.parse(part)[0]
        new_part = (morph_part.inflect({case})[0].capitalize() if morph_part
                    .inflect({case}) else part)
        new_name += f' {new_part}'
    return new_name.strip()

In [76]:
print(case_name('Дмитрий Сергеевич', 'gent'), 'genetive')
print(case_name('Дмитрия Сергеевича', 'nomn'), 'nominative')
print(case_name('Дмитрий Сергеевич', 'datv'), 'dative')
print(case_name('Дмитрий Сергеевич', 'accs'), 'accusative')

Дмитрия Сергеевича genetive
Дмитрий Сергеевич nominative
Дмитрию Сергеевичу dative
Дмитрия Сергеевича accusative


**У Spacy и Pymorphy разная разметка падежей, самым быстрым решением показалось сделать маппинг.**

In [77]:
case_mapping = {'Nom': 'nomn',
                'Gen': 'gent',
                'Dat': 'datv',
                'Acc': 'accs',
                'Abl': 'ablt',
                'Loc': 'loct',
               }

**Функция для поиска падежа с помощью Pymorphy. Принимает на вход объект токена Spacy и словарь с маппингом Spacy-Pymorphy. В синтаксическом дереве предложения ищется либо главное слово, либо ближайшее зависимое к аннотации MASK. Возвращает падеж главного слова, либо ближайшего потомка, либо, если вдруг их нет, дательный падеж.**

In [78]:
def find_case(token, case_mapping):
    childs_cases = [child.morph.to_dict().get('Case') for child in
                    token.children if child.morph.to_dict().get('Case')]
    case = ''
    if token.head.morph.to_dict().get('Case'):
        case = token.head.morph.to_dict().get('Case')
    elif len(childs_cases) > 0:
        case = childs_cases[-1]
        
    if case_mapping.get(case):
        case = case_mapping.get(case)
    else:
        case = 'datv'
    return case

**Затем идет перебор всего текста, очистка от двойных токенов MASK, множественных пробелов, и замена токенов MASK на именованную сущность в соответствующем падеже.**

In [79]:
def change_masks(full_name, text, case_mapping):
    new_text = ''
    text = re.sub("(MASK ?)+", 'MASK', text)
    text = re.sub(" +", ' ', text)
    doc = nlp(text)
    for token in doc:
        if 'MASK' in token.text:
            my_case = find_case(token, case_mapping)
            new_token = case_name(full_name, my_case)
        else:
            new_token = token.text
        new_token = f' {new_token}' if new_token.isalpha() else new_token
        new_text += new_token
    return new_text.strip()

**Все собирается в функцию для генерации текста**

In [80]:
def generate_congrat(full_name):
    text = local_generator(full_name)[0]['generated_text']
    final_text = change_masks(full_name, text, case_mapping)
    return final_text

In [81]:
generate_congrat('Александру')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Александру Александровичу Моисееву, народному артисту России Уважаемый Александру Вас с-летием. Вы осуществили свою заветнуюмечту, стали известным актёром и режиссёром, заслужили высокое профессиональноепризнание и искреннюю зрительскую любовь. И конечно, во многом благодаря Александру потенциалу, незаурядным лидерским качествам, умениюсплотить коллектив театра вокруг важных, востребованных задач, стоящих перед знаменитым Александра, и в целом за счёт привлечения в творческийобщественный интерес талантливых, увлечённыхлюдей, для того, чтобы приобщиться к прекрасному. Желаю Вам здоровья, успехов и всего наилучшего. Александру. Александру, художественному руководителю Александру» Уважаемый Александра поздравляю Вас с-летием. Вас по праву считают одним из наиболееталантливых и известных российских актёров, достойным представителемлегендарной плеяды« ленкомовцев». Ваши работы в кинематографе и на сцене родного театра неизменно становятся большим событием и надолго остаютсяв памяти благода

In [82]:
generate_congrat('Александру')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Александру Александровичу Моисееву, народному артисту России Уважаемый Александру Вас с-летием. Вы осуществили свою заветнуюмечту, стали известным актёром и режиссёром, заслужили высокое профессиональноепризнание и искреннюю зрительскую любовь. И конечно, во многом благодаря Александру потенциалу, незаурядным лидерским качествам, умениюсплотить коллектив театра вокруг важных, востребованных задач, стоящих перед знаменитым Александра, и в целом за счёт привлечения в творческийобщественный интерес талантливых, увлечённыхлюдей, для того, чтобы приобщиться к прекрасному. Желаю Вам здоровья, успехов и всего наилучшего. Александру. Александру, художественному руководителю Александру» Уважаемый Александра поздравляю Вас с-летием. Вас по праву считают одним из наиболееталантливых и известных российских актёров, достойным представителемлегендарной плеяды« ленкомовцев». Ваши работы в кинематографе и на сцене родного театра неизменно становятся большим событием и надолго остаютсяв памяти благода

In [83]:
generate_congrat('Сергею')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Сергею Сергею» Сердечно поздравляю Вас с-летием. Вы осуществили свою мечту– состоялись в сложной, многогранной профессии актёра, блестяще раскрыли свой яркий, самобытный талант в театре и кинематографе. Богатый сценический опыт и неизменная приверженность традициям знаменитой вахтанговской школы на протяжении долгих лет помогают Вам руководить коллективом театра и добиваться успеха, неизменно добиваться успеха. Желаю Вам здоровья, благополучия и всего самого доброго. Сергею. Сергея, Сергею, Сергею, Сергею, Сергею, женской сборной России, победившей на Всемирной летней универсиаде года в Тайбэе в соревнованиях групп по художественной гимнастике Уважаемые Сергея, Сергею, Сергею, Сергею, Сергею, Сергею вас с большим, заслуженным успехом. В борьбе с сильнейшими командами мира вы показали блестящийрезультат, настоящую сплочённость и твёрдый настрой на победу, по праву сталичемпионками и поднялись на высшую ступень пьедестала почёта в упражнениях с булавами Сергею, Сергею, Сергею вам новых 

In [84]:
generate_congrat('Сергею')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Сергею Сергею» Сердечно поздравляю Вас с-летием. Вы осуществили свою мечту– состоялись в сложной, многогранной профессии актёра, блестяще раскрыли свой яркий, самобытный талант в театре и кинематографе. Богатый сценический опыт и неизменная приверженность традициям знаменитой вахтанговской школы на протяжении долгих лет помогают Вам руководить коллективом театра и добиваться успеха, неизменно добиваться успеха. Желаю Вам здоровья, благополучия и всего самого доброго. Сергею. Сергея, Сергею, Сергею, Сергею, Сергею, женской сборной России, победившей на Всемирной летней универсиаде года в Тайбэе в соревнованиях групп по художественной гимнастике Уважаемые Сергея, Сергею, Сергею, Сергею, Сергею, Сергею вас с большим, заслуженным успехом. В борьбе с сильнейшими командами мира вы показали блестящийрезультат, настоящую сплочённость и твёрдый настрой на победу, по праву сталичемпионками и поднялись на высшую ступень пьедестала почёта в упражнениях с булавами Сергею, Сергею, Сергею вам новых 

In [85]:
generate_congrat('Алексею')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Алексею Моисееву, народному артисту России Уважаемый Алексею поздравления с-летним юбилеем. Вас по праву считают выдающимся режиссёром и настоящим Алексею, обогатившим отечественный и мировой кинематограф своими талантливыми, содержательными работами. Ваши фильмы смело ломают устоявшиеся стереотипы, заставляют зрителей по- новому взглянуть на, казалось бы, привычный мир. И потому они всегда становятся заметным событием в культурной жизни. Желаю Вам здоровья, реализации намеченных планов и всего наилучшего. Алексею. Алексею, победительнице чемпионата мира по спортивной борьбе года в Будапеште в соревнованиях по греко- римской борьбе в весовой категории до кг Алексея поздравить Вас с победой на чемпионате мира по спортивной борьбе года в Будапеште. В борьбе с сильными, достойными соперницами Вы проявили высокие требования к себе, представили нашу страну на соревнованиях по спортивной борьбе, продемонстрировали отличную подготовку, твёрдый настрой на победу. Желаю Вам новых спортивных до

In [86]:
generate_congrat('Алексею')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Алексею Моисееву, народному артисту России Уважаемый Алексею поздравления с-летним юбилеем. Вас по праву считают выдающимся режиссёром и настоящим Алексею, обогатившим отечественный и мировой кинематограф своими талантливыми, содержательными работами. Ваши фильмы смело ломают устоявшиеся стереотипы, заставляют зрителей по- новому взглянуть на, казалось бы, привычный мир. И потому они всегда становятся заметным событием в культурной жизни. Желаю Вам здоровья, реализации намеченных планов и всего наилучшего. Алексею. Алексею, победительнице чемпионата мира по спортивной борьбе года в Будапеште в соревнованиях по греко- римской борьбе в весовой категории до кг Алексея поздравить Вас с победой на чемпионате мира по спортивной борьбе года в Будапеште. В борьбе с сильными, достойными соперницами Вы проявили высокие требования к себе, представили нашу страну на соревнованиях по спортивной борьбе, продемонстрировали отличную подготовку, твёрдый настрой на победу. Желаю Вам новых спортивных до

In [87]:
generate_congrat('Виктор')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'ВикторАлександрович!Поздравляю Вас с-летним юбилеем. Фундаментальные труды и крупные открытия в области физической химии принесли Вам заслуженное признание и высокий авторитет в научном сообществе. Ваши разработки служат научно- техническому прогрессу, они востребованы в самых разных отраслях отечественной промышленности. Отрадно, что плодотворную исследовательскую деятельность Вы успешно совмещаете с активной организационной и педагогической работой, щедро делитесь своими знаниями и опытом с талантливой молодежью. Здоровья Вам, благополучия и новых успехов. Виктору. Коллективу Виктору Уважаемыедрузья!Поздравляю вас с-летием Виктору дерматовенерологии и косметологии имениН.Н.Приорова. Юбилей вашего прославленного центра– большоеи радостное событие в общественной, культурной, спортивной жизни страны, в истории отечественной медицины. Вы по праву можете гордиться многими поколениями талантливых специалистов, которые вместе с Вами создавали и внедряли в практику уникальные методики и тех

In [88]:
generate_congrat('Виктор')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'ВикторАлександрович!Поздравляю Вас с-летним юбилеем. Фундаментальные труды и крупные открытия в области физической химии принесли Вам заслуженное признание и высокий авторитет в научном сообществе. Ваши разработки служат научно- техническому прогрессу, они востребованы в самых разных отраслях отечественной промышленности. Отрадно, что плодотворную исследовательскую деятельность Вы успешно совмещаете с активной организационной и педагогической работой, щедро делитесь своими знаниями и опытом с талантливой молодежью. Здоровья Вам, благополучия и новых успехов. Виктору. Коллективу Виктору Уважаемыедрузья!Поздравляю вас с-летием Виктору дерматовенерологии и косметологии имениН.Н.Приорова. Юбилей вашего прославленного центра– большоеи радостное событие в общественной, культурной, спортивной жизни страны, в истории отечественной медицины. Вы по праву можете гордиться многими поколениями талантливых специалистов, которые вместе с Вами создавали и внедряли в практику уникальные методики и тех

In [89]:
generate_congrat('Егору')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Егору театра и кино, народному артисту СССР Егору поздравляю Вас с Днём рождения. С самых первых, дебютных работ в кинематографе Вы задали высочайшуюпрофессиональную планку и удерживаете её на протяжении всей своей насыщеннойактёрской карьеры. Особого уважения заслуживает Ваша активная жизненная и гражданская позиция, большой личный вклад в реализацию общественно значимыхпросветительских, патриотических инициатив, в сохранение исторической памятинашего народа. Желаю Вам крепкого здоровья, неиссякаемой энергии и всего наилучшего. Егору. Сотрудникам и ветеранам органов прокуратуры России Уважаемыетоварищи!Поздравляю вас с профессиональным праздником и-йгодовщиной создания органов прокуратуры в России. Сменялись исторические события и эпохи, но защитаинтересов государства, общества, граждан всегда оставалась важнейшимприоритетом деятельности прокуратуры. Сегодня мы чествуем тех, кто ежедневным добросовестнымтрудом приумножает традиции бескомпромиссного служения закону, заложенныелучшими 

In [90]:
generate_congrat('Егору')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Егору театра и кино, народному артисту СССР Егору поздравляю Вас с Днём рождения. С самых первых, дебютных работ в кинематографе Вы задали высочайшуюпрофессиональную планку и удерживаете её на протяжении всей своей насыщеннойактёрской карьеры. Особого уважения заслуживает Ваша активная жизненная и гражданская позиция, большой личный вклад в реализацию общественно значимыхпросветительских, патриотических инициатив, в сохранение исторической памятинашего народа. Желаю Вам крепкого здоровья, неиссякаемой энергии и всего наилучшего. Егору. Сотрудникам и ветеранам органов прокуратуры России Уважаемыетоварищи!Поздравляю вас с профессиональным праздником и-йгодовщиной создания органов прокуратуры в России. Сменялись исторические события и эпохи, но защитаинтересов государства, общества, граждан всегда оставалась важнейшимприоритетом деятельности прокуратуры. Сегодня мы чествуем тех, кто ежедневным добросовестнымтрудом приумножает традиции бескомпромиссного служения закону, заложенныелучшими 

In [91]:
generate_congrat('Семен')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'СеменСемизанович!Примите поздравления с-летним юбилеем. Высокий профессионализм, поистине неисчерпаемая энергия и уважительное, доброжелательное отношение к зрителям на протяжении многих лет помогают Вам руководить Семёну. И конечно, одной из жемчужин Вашего коллектива является« Семён», отмечающий своё‑летие. Поздравляю Вас с этимзамечательным, праздничным событием. У истоков создания« Семёна» стояли выдающиеся представители отечественнойкультуры, известные учёные и конструкторы. Их разработки существенно повысили боеспособность нашей армии во время Великой Отечественной войны. Сегодня под Вашим руководством создан целый ряд новых зенитных и противотанковых ракетных комплексов, которые успешно применяются в различных отраслях отечественной армии, а также в качестве переносных зенитных и противотанковых ракетныхкомплексов.реабилитационные технологии, которые успешно применяются в производстве ракетных комплексов нового поколения. Уверен, что Ваши усилия на посту руководителя Семёна и в

In [92]:
generate_congrat('Семен')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'СеменСемизанович!Примите поздравления с-летним юбилеем. Высокий профессионализм, поистине неисчерпаемая энергия и уважительное, доброжелательное отношение к зрителям на протяжении многих лет помогают Вам руководить Семёну. И конечно, одной из жемчужин Вашего коллектива является« Семён», отмечающий своё‑летие. Поздравляю Вас с этимзамечательным, праздничным событием. У истоков создания« Семёна» стояли выдающиеся представители отечественнойкультуры, известные учёные и конструкторы. Их разработки существенно повысили боеспособность нашей армии во время Великой Отечественной войны. Сегодня под Вашим руководством создан целый ряд новых зенитных и противотанковых ракетных комплексов, которые успешно применяются в различных отраслях отечественной армии, а также в качестве переносных зенитных и противотанковых ракетныхкомплексов.реабилитационные технологии, которые успешно применяются в производстве ракетных комплексов нового поколения. Уверен, что Ваши усилия на посту руководителя Семёна и в

In [93]:
generate_congrat('Александре')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Александре Моисеевичевых, ансамбль по праву считается ярким, неординарнымявлением в отечественной и мировой культуре, нашим подлинным национальнымдостоянием. Для Александру был не только любимым детищем, но делом и смыслом всей жизни. Благодаря его подвижническому, вдохновенному труду и созидательной энергии, а также таланту и безупречному мастерству артистов Александра высочайшеепрофессиональное признание, покорил миллионы зрителей как в нашей стране, так и далеко за её пределами. Важно, что вы бережно храните бесценное творческое, духовное наследие Мастера и сложившиеся в коллективе традиции. И потому вашимконцертникам всегда сопутствует успех. Желаю вам новых творческих удач, вдохновения и всего самого доброго. Александру. Государственным служащим, работникам и ветеранам Александру вас с профессиональным праздником– Днём судебного пристава. На протяжении многих лет Александра ключевую роль в осуществлениигосударственного контроля за эффективным использованием средств федеральногобю

In [94]:
generate_congrat('Александре')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Александре Моисеевичевых, ансамбль по праву считается ярким, неординарнымявлением в отечественной и мировой культуре, нашим подлинным национальнымдостоянием. Для Александру был не только любимым детищем, но делом и смыслом всей жизни. Благодаря его подвижническому, вдохновенному труду и созидательной энергии, а также таланту и безупречному мастерству артистов Александра высочайшеепрофессиональное признание, покорил миллионы зрителей как в нашей стране, так и далеко за её пределами. Важно, что вы бережно храните бесценное творческое, духовное наследие Мастера и сложившиеся в коллективе традиции. И потому вашимконцертникам всегда сопутствует успех. Желаю вам новых творческих удач, вдохновения и всего самого доброго. Александру. Государственным служащим, работникам и ветеранам Александру вас с профессиональным праздником– Днём судебного пристава. На протяжении многих лет Александра ключевую роль в осуществлениигосударственного контроля за эффективным использованием средств федеральногобю

In [95]:
generate_congrat('Завод "Салют"')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Завод" Салют" имениЗавода "Салют", за прошедшие годы введены в строй тысячи километров современных автомагистралей, других транспортных объектов, отвечающих мировым стандартам безопасности, активно модернизируется дорожная инфраструктура в регионах страны. И конечно, ярким примером вашей отличной работы и высокого мастерства служат поистине уникальные стройки: мосты, тоннели, транспортные развязки, возведённые в преддверии саммитаАТЭС- во Владивостоке и Олимпиады года в Сочи. Уверен, вы и впредь будете беречь и приумножать замечательные профессиональные традиции, достойно решать поставленные задачи на благо России и её граждан.Заводу "Салют". Творческому коллективуЗаводу "Салют" друзья! Примите поздравления по случаю-летия со дня образованияЗавода "Салют". За прошедшие годы ваш коллектив стал авторитетным творческим коллективом, многое сделал для развития отечественного искусства. Вас ценят и как замечательного педагога, более полувека возглавляющего знаменитыйЗавода "Салют". И конечн

In [96]:
generate_congrat('Завод "Салют"')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Завод" Салют" имениЗавода "Салют", за прошедшие годы введены в строй тысячи километров современных автомагистралей, других транспортных объектов, отвечающих мировым стандартам безопасности, активно модернизируется дорожная инфраструктура в регионах страны. И конечно, ярким примером вашей отличной работы и высокого мастерства служат поистине уникальные стройки: мосты, тоннели, транспортные развязки, возведённые в преддверии саммитаАТЭС- во Владивостоке и Олимпиады года в Сочи. Уверен, вы и впредь будете беречь и приумножать замечательные профессиональные традиции, достойно решать поставленные задачи на благо России и её граждан.Заводу "Салют". Творческому коллективуЗаводу "Салют" друзья! Примите поздравления по случаю-летия со дня образованияЗавода "Салют". За прошедшие годы ваш коллектив стал авторитетным творческим коллективом, многое сделал для развития отечественного искусства. Вас ценят и как замечательного педагога, более полувека возглавляющего знаменитыйЗавода "Салют". И конечн

In [97]:
generate_congrat('Компания "Звезда"')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Компания" Звезда" по праву считается одной из ключевых, стратегически важныхорганизаций отечественной экономики, признанным лидером в производстве оборудования для нефтегазовой, судостроительной, энергетической, других стратегических отраслей отечественной экономики. Здесьтрудятся опытные профессионалы, которые активно внедряют инновационныетехнологии, передовые управленческие и инженерные решения, ставят перед собойбольшие, амбициозные цели и добиваются успеха. Важно, что, совершенствуя производственную базу, привлекая крупныеинвестиции, создавая современные рабочие места, вы уделяете приоритетноевнимание развитию регионов, деятельно участвуете в общественной, спортивной, культурной жизни Челябинской области, Урала, всей страны. Рассчитываю, что, опираясь на богатый опыт и замечательныетрадиции, вы и впредь будете добросовестно трудиться, воплощать в жизнь востребованные творческие проекты, добиваться намеченных планов, добиваться намеченных целей на благоРоссийского уровня. Желаю ва

In [98]:
generate_congrat('Компания "Звезда"')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Компания" Звезда" по праву считается одной из ключевых, стратегически важныхорганизаций отечественной экономики, признанным лидером в производстве оборудования для нефтегазовой, судостроительной, энергетической, других стратегических отраслей отечественной экономики. Здесьтрудятся опытные профессионалы, которые активно внедряют инновационныетехнологии, передовые управленческие и инженерные решения, ставят перед собойбольшие, амбициозные цели и добиваются успеха. Важно, что, совершенствуя производственную базу, привлекая крупныеинвестиции, создавая современные рабочие места, вы уделяете приоритетноевнимание развитию регионов, деятельно участвуете в общественной, спортивной, культурной жизни Челябинской области, Урала, всей страны. Рассчитываю, что, опираясь на богатый опыт и замечательныетрадиции, вы и впредь будете добросовестно трудиться, воплощать в жизнь востребованные творческие проекты, добиваться намеченных планов, добиваться намеченных целей на благоРоссийского уровня. Желаю ва

In [99]:
generate_congrat('Сообщество "Вымпел"')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Сообщество" Вымпел" всегда славилась своим спектромсоревнований и триумфом. И сегодня«Сообществу "Вымпел"» по праву считается одной из наиболее успешных и популярныхсценических площадок столицы. Здесь всегда царитатмосфера свободы, созидания и творчества. Отрадно, что сегодня под эгидой«Сообществу "Вымпел"» воплощается в жизнь большое количество информационных и развлекательныхпроектов, которые вызывают заинтересованное внимание самой широкой аудитории, ведётся неустанная работа над совершенствованием технологий и продуктовмедиарынка. И конечно, отмечу большой вклад«Сообщество "Вымпел"» в развитие детской литературы, популяризацию чтения среди юных граждан нашейстраны. Желаю вам успехов и всего наилучшего.Сообществу "Вымпел". Сотрудникам и ветеранам«Сообществу "Вымпел"» Дорогиедрузья!Поздравляю вас с-летием создания«Сообщества "Вымпел". Уверен, что свой юбилей вы отмечаете яркой и интересной творческой среды, успехов и всего наилучшего. И конечно, отмечу, что по случаю-летия со дня ег

In [100]:
generate_congrat('Сообщество "Вымпел"')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Сообщество" Вымпел" всегда славилась своим спектромсоревнований и триумфом. И сегодня«Сообществу "Вымпел"» по праву считается одной из наиболее успешных и популярныхсценических площадок столицы. Здесь всегда царитатмосфера свободы, созидания и творчества. Отрадно, что сегодня под эгидой«Сообществу "Вымпел"» воплощается в жизнь большое количество информационных и развлекательныхпроектов, которые вызывают заинтересованное внимание самой широкой аудитории, ведётся неустанная работа над совершенствованием технологий и продуктовмедиарынка. И конечно, отмечу большой вклад«Сообщество "Вымпел"» в развитие детской литературы, популяризацию чтения среди юных граждан нашейстраны. Желаю вам успехов и всего наилучшего.Сообществу "Вымпел". Сотрудникам и ветеранам«Сообществу "Вымпел"» Дорогиедрузья!Поздравляю вас с-летием создания«Сообщества "Вымпел". Уверен, что свой юбилей вы отмечаете яркой и интересной творческой среды, успехов и всего наилучшего. И конечно, отмечу, что по случаю-летия со дня ег

In [101]:
generate_congrat('Мудрейший утконос')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Мудрейший утконосимый вклад в развитие российско- итальянских отношений, их вывод на уровень стратегического партнёрства. Уверен, что конструктивное взаимодействие между нашими странами, характеризующееся впечатляющими результатами во внешнеполитической, торгово- экономической и культурно- гуманитарной сферах, будет и далее весомым фактором поддержания долгосрочной стабильности в Европе и в мире в целом. Искренне желаю Вам, уважаемыйМудрейшего Утконоса, крепкого здоровья, счастья, благополучия и успехов в столь ответственной деятельности во главе государства. С уважением,Мудрейшему Утконосу.Мудрейшему Утконосу, актрисе театра и кино, народной артистке СССР, лауреатуМудрейшему Утконосу поздравления с юбилейным Днём рождения. Вас, замечательную актрису, знают по глубоким, многогранным театральным работам и по ярким, запоминающимся, поистине знаковым ролям в кинематографе. Наградой за Ваш щедрый талант, удивительную искренность и обаяние снискали Вам любовь зрителей и уважение коллег. Ва

In [102]:
generate_congrat('Мудрейший утконос')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Мудрейший утконосимый вклад в развитие российско- итальянских отношений, их вывод на уровень стратегического партнёрства. Уверен, что конструктивное взаимодействие между нашими странами, характеризующееся впечатляющими результатами во внешнеполитической, торгово- экономической и культурно- гуманитарной сферах, будет и далее весомым фактором поддержания долгосрочной стабильности в Европе и в мире в целом. Искренне желаю Вам, уважаемыйМудрейшего Утконоса, крепкого здоровья, счастья, благополучия и успехов в столь ответственной деятельности во главе государства. С уважением,Мудрейшему Утконосу.Мудрейшему Утконосу, актрисе театра и кино, народной артистке СССР, лауреатуМудрейшему Утконосу поздравления с юбилейным Днём рождения. Вас, замечательную актрису, знают по глубоким, многогранным театральным работам и по ярким, запоминающимся, поистине знаковым ролям в кинематографе. Наградой за Ваш щедрый талант, удивительную искренность и обаяние снискали Вам любовь зрителей и уважение коллег. Ва

In [103]:
generate_congrat('Заморские гнилые')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Заморские гнилые дороги, прочные отношения между нашими странами. Убеждён, что дальнейшее наращивание всего комплекса взаимовыгодных российско- туркменских связей в полной мере отвечает интересам наших народов, идёт в русле упрочения стабильности и безопасности на европейском континенте. Желаю Вам крепкого здоровья и успехов, а всем гражданам Туркменистана– благополучия и процветания.Заморским Гнилым. КоллективуЗаморским Гнилымдрузья!Искренне поздравляю вас с-летием образованияЗаморских Гнилых прошедшие годы ваш вуз заслужил репутацию авторитетного учебного и научно- исследовательского центра страны. В нём преподавали выдающиеся учёные и конструкторы, среди которых–Заморских ГнилыхЗаморским Гнилым. Были подготовлены несколько поколений высококвалифицированных специалистов, среди которых– известные учёные и конструкторы, лауреаты Государственных премий СССР и России. Были подготовлены несколько поколений высококвалифицированных специалистов, среди которых–Заморских ГнилыхЗаморским Гнил

In [104]:
generate_congrat('Заморские гнилые')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Заморские гнилые дороги, прочные отношения между нашими странами. Убеждён, что дальнейшее наращивание всего комплекса взаимовыгодных российско- туркменских связей в полной мере отвечает интересам наших народов, идёт в русле упрочения стабильности и безопасности на европейском континенте. Желаю Вам крепкого здоровья и успехов, а всем гражданам Туркменистана– благополучия и процветания.Заморским Гнилым. КоллективуЗаморским Гнилымдрузья!Искренне поздравляю вас с-летием образованияЗаморских Гнилых прошедшие годы ваш вуз заслужил репутацию авторитетного учебного и научно- исследовательского центра страны. В нём преподавали выдающиеся учёные и конструкторы, среди которых–Заморских ГнилыхЗаморским Гнилым. Были подготовлены несколько поколений высококвалифицированных специалистов, среди которых– известные учёные и конструкторы, лауреаты Государственных премий СССР и России. Были подготовлены несколько поколений высококвалифицированных специалистов, среди которых–Заморских ГнилыхЗаморским Гнил

In [ ]:
a = torch.zeros(4, 4)

In [ ]:
a = a + 1.1

In [ ]:
b = a.cuda()

In [ ]:
b**2